In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Chronic_Fatigue_Syndrome"
cohort = "GSE67311"

# Input paths
in_trait_dir = "../../input/GEO/Chronic_Fatigue_Syndrome"
in_cohort_dir = "../../input/GEO/Chronic_Fatigue_Syndrome/GSE67311"

# Output paths
out_data_file = "../../output/preprocess/Chronic_Fatigue_Syndrome/GSE67311.csv"
out_gene_data_file = "../../output/preprocess/Chronic_Fatigue_Syndrome/gene_data/GSE67311.csv"
out_clinical_data_file = "../../output/preprocess/Chronic_Fatigue_Syndrome/clinical_data/GSE67311.csv"
json_path = "../../output/preprocess/Chronic_Fatigue_Syndrome/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Peripheral Blood Gene Expression in Fibromyalgia Patients Reveals  Potential Biological Markers and Physiological Pathways"
!Series_summary	"Fibromyalgia (FM) is a common pain disorder characterized by dysregulation in the processing of pain. Although FM has similarities with other rheumatologic pain disorders, the search for objective markers has not been successful. In the current study we analyzed gene expression in the whole blood of 70 fibromyalgia patients and 70 healthy matched controls. Global molecular profiling revealed an upregulation of several inflammatory molecules in FM patients and downregulation of specific pathways related to hypersensitivity and allergy. There was a differential expression of genes in known pathways for pain processing, such as glutamine/glutamate signaling and axonal development. We also identified a panel of candidate gene expression-based classifiers that could establish an objective blood-based molecular dia

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# This is likely gene expression data as it mentions "Blood global gene expression profiling" and
# "Affymetrix® Human Gene 1.1 ST Peg arrays" in the background information
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 For trait: Chronic Fatigue Syndrome
# From the sample characteristics dictionary, we can see row 8 contains information about CFS
trait_row = 8

# 2.2 Trait conversion function
def convert_trait(value):
    if pd.isna(value):
        return None
    if isinstance(value, str):
        value = value.lower()
        if "yes" in value:
            return 1
        elif "no" in value:
            return 0
        else:
            return None
    return None

# Age is not available in the dataset
age_row = None
def convert_age(value):
    return None

# Gender is not available in the dataset
gender_row = None
def convert_gender(value):
    return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Look for the sample characteristics file in the cohort directory
    sample_char_file = os.path.join(in_cohort_dir, "sample_characteristics.csv")
    
    if os.path.exists(sample_char_file):
        clinical_data = pd.read_csv(sample_char_file)
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data, 
            trait=trait, 
            trait_row=trait_row, 
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the dataframe
        print("Preview of selected clinical features:")
        print(preview_df(selected_clinical_df))
        
        # Save to CSV
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    else:
        print(f"Sample characteristics file not found at {sample_char_file}")
        print("Cannot extract clinical features without sample data.")


Sample characteristics file not found at ../../input/GEO/Chronic_Fatigue_Syndrome/GSE67311/sample_characteristics.csv
Cannot extract clinical features without sample data.


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 66
Header line: "ID_REF"	"GSM1644447"	"GSM1644448"	"GSM1644449"	"GSM1644450"	"GSM1644451"	"GSM1644452"	"GSM1644453"	"GSM1644454"	"GSM1644455"	"GSM1644456"	"GSM1644457"	"GSM1644458"	"GSM1644459"	"GSM1644460"	"GSM1644461"	"GSM1644462"	"GSM1644463"	"GSM1644464"	"GSM1644465"	"GSM1644466"	"GSM1644467"	"GSM1644468"	"GSM1644469"	"GSM1644470"	"GSM1644471"	"GSM1644472"	"GSM1644473"	"GSM1644474"	"GSM1644475"	"GSM1644476"	"GSM1644477"	"GSM1644478"	"GSM1644479"	"GSM1644480"	"GSM1644481"	"GSM1644482"	"GSM1644483"	"GSM1644484"	"GSM1644485"	"GSM1644486"	"GSM1644487"	"GSM1644488"	"GSM1644489"	"GSM1644490"	"GSM1644491"	"GSM1644492"	"GSM1644493"	"GSM1644494"	"GSM1644495"	"GSM1644496"	"GSM1644497"	"GSM1644498"	"GSM1644499"	"GSM1644500"	"GSM1644501"	"GSM1644502"	"GSM1644503"	"GSM1644504"	"GSM1644505"	"GSM1644506"	"GSM1644507"	"GSM1644508"	"GSM1644509"	"GSM1644510"	"GSM1644511"	"GSM1644512"	"GSM1644513"	"GSM1644514"	"GSM1644515"	"GSM1644516"	"GSM1644517"	"GSM1644518"	"GSM1644519"	

Index(['7892501', '7892502', '7892503', '7892504', '7892505', '7892506',
       '7892507', '7892508', '7892509', '7892510', '7892511', '7892512',
       '7892513', '7892514', '7892515', '7892516', '7892517', '7892518',
       '7892519', '7892520'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The identifiers in the gene expression data appear to be probe IDs (like "7892501"), not human gene symbols
# These are likely to be Illumina microarray probe IDs that need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['7896736', '7896738', '7896740', '7896742', '7896744'], 'GB_LIST': [nan, nan, 'NM_001005240,NM_001004195,NM_001005484,BC136848,BC136907', 'BC118988,AL137655', 'NM_001005277,NM_001005221,NM_001005224,NM_001005504,BC137547'], 'SPOT_ID': ['chr1:53049-54936', 'chr1:63015-63887', 'chr1:69091-70008', 'chr1:334129-334296', 'chr1:367659-368597'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': [53049.0, 63015.0, 69091.0, 334129.0, 367659.0], 'RANGE_STOP': [54936.0, 63887.0, 70008.0, 334296.0, 368597.0], 'total_probes': [7.0, 31.0, 24.0, 6.0, 36.0], 'gene_assignment': ['---', '---', 'NM_001005240 // OR4F17 // olfactory receptor, family 4, subfamily F, member 17 // 19p13.3 // 81099 /// NM_001004195 // OR4F4 // olfactory receptor, family 4, subfamily F, member 4 // 15q26.3 // 26682 /// NM_001005484 // OR4F5 //

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Ensure we have the gene expression data loaded properly
gene_data = get_genetic_data(matrix_file)
print(f"Gene expression data shape: {gene_data.shape}")
print(f"First few probe IDs: {list(gene_data.index[:5])}")

# 2. Examine the gene annotation structure and prepare the mapping
print("Creating gene mapping from probe IDs to gene symbols...")

# Check the relationship between gene expression probe IDs and annotation IDs
gene_ids = set(gene_data.index.astype(str))
annotation_ids = set(gene_annotation['ID'].astype(str))
overlap = gene_ids.intersection(annotation_ids)
print(f"\nProbe ID overlap: {len(overlap)} out of {len(gene_ids)} expression probes")

# Create a mapping dataframe, focusing on IDs that appear in our gene expression data
mapping_df = gene_annotation[gene_annotation['ID'].astype(str).isin(gene_ids)][['ID', 'gene_assignment']].copy()
print(f"Filtered mapping dataframe contains {len(mapping_df)} rows")

# Extract gene symbols from gene assignments
print("\nExtracting gene symbols from gene assignments...")
mapping_df['Gene'] = mapping_df['gene_assignment'].apply(extract_human_gene_symbols)

# Print a sample of the extracted gene symbols
print("\nSample of extracted gene symbols (first 5 rows):")
print(mapping_df[['ID', 'Gene']].head(5))

# Count how many mapped probes have gene assignments
mappable_probes = mapping_df[mapping_df['Gene'].apply(len) > 0].shape[0]
print(f"\nNumber of probes with gene mappings: {mappable_probes} out of {mapping_df.shape[0]} total probes")

# Check if we have any valid mappings
if mappable_probes == 0:
    print("\nWARNING: No valid gene mappings found. Using a fallback approach...")
    # Fallback: Try exact string matching for probe IDs
    mapping_df = gene_annotation[['ID', 'gene_assignment']].copy()
    mapping_df['ID'] = mapping_df['ID'].astype(str)
    mapping_df['Gene'] = mapping_df['gene_assignment'].apply(extract_human_gene_symbols)
    # Only keep rows with extracted gene symbols
    mapping_df = mapping_df[mapping_df['Gene'].apply(len) > 0]
    print(f"Fallback mapping contains {len(mapping_df)} rows with gene symbols")

# Apply gene mapping to get gene expression data
print("\nApplying gene mapping to convert probe data to gene expression data...")
gene_data_mapped = apply_gene_mapping(gene_data, mapping_df[['ID', 'Gene']])
print(f"Mapped gene expression data shape: {gene_data_mapped.shape}")

# If we still don't have any genes, try a more direct approach
if gene_data_mapped.shape[0] == 0:
    print("\nWARNING: Still no mapped genes. Attempting direct mapping of most common genes...")
    # Direct mapping approach
    import re
    
    # We'll create a simple mapping from each probe to potential genes
    direct_mappings = {}
    for idx, row in gene_annotation.iterrows():
        probe_id = str(row['ID'])
        gene_text = str(row['gene_assignment'])
        # Extract gene symbols - common pattern: SYMBOL // DESCRIPTION
        matches = re.findall(r'(\w+) // ', gene_text)
        if matches:
            direct_mappings[probe_id] = matches
    
    # Create a new mapping dataframe
    direct_map_records = []
    for probe_id, genes in direct_mappings.items():
        for gene in genes:
            if gene not in ['NM', 'BC', 'ENST', 'NC'] and len(gene) > 1:  # Filter out common prefixes
                direct_map_records.append({'ID': probe_id, 'Gene': gene})
    
    if direct_map_records:
        direct_mapping_df = pd.DataFrame(direct_map_records)
        print(f"Created direct mapping with {len(direct_mapping_df)} probe-to-gene mappings")
        gene_data_mapped = apply_gene_mapping(gene_data, direct_mapping_df)
        print(f"Direct mapped gene expression data shape: {gene_data_mapped.shape}")

# Normalize gene symbols using standard function or keep as is if still empty
if gene_data_mapped.shape[0] > 0:
    print("\nNormalizing gene symbols...")
    gene_data = normalize_gene_symbols_in_index(gene_data_mapped)
    print(f"Final gene data shape after normalization: {gene_data.shape}")
else:
    print("\nWARNING: Could not map any genes. Creating an empty gene expression dataset.")
    gene_data = pd.DataFrame(columns=gene_data.columns)

# Print information about the resulting gene expression data
print(f"\nFinal gene expression data shape: {gene_data.shape}")
if gene_data.shape[0] > 0:
    print(f"First 10 gene symbols: {list(gene_data.index[:10])}")
else:
    print("WARNING: No genes were found in the final dataset!")

# Save the gene expression data to a CSV file
print("\nSaving gene expression data to CSV file...")
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene expression data shape: (33297, 142)
First few probe IDs: ['7892501', '7892502', '7892503', '7892504', '7892505']
Creating gene mapping from probe IDs to gene symbols...

Probe ID overlap: 33297 out of 33297 expression probes


Filtered mapping dataframe contains 4761471 rows

Extracting gene symbols from gene assignments...



Sample of extracted gene symbols (first 5 rows):
        ID                                        Gene
0  7896736                                          []
1  7896738                                          []
2  7896740  [OR4F17, OR4F4, OR4F5, BC136848, BC136907]
3  7896742    [SEPT14, BC118988, NCRNA00266, AL137655]
4  7896744   [OR4F16, OR4F29, OR4F3, OR4F21, BC137547]



Number of probes with gene mappings: 22011 out of 4761471 total probes

Applying gene mapping to convert probe data to gene expression data...


Mapped gene expression data shape: (0, 142)



Created direct mapping with 447055 probe-to-gene mappings


Direct mapped gene expression data shape: (55006, 142)

Normalizing gene symbols...
Final gene data shape after normalization: (19854, 142)

Final gene expression data shape: (19854, 142)
First 10 gene symbols: ['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS', 'AADAC']

Saving gene expression data to CSV file...


Gene expression data saved to ../../output/preprocess/Chronic_Fatigue_Syndrome/gene_data/GSE67311.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Create clinical features directly from clinical_data using the conversion functions defined earlier
clinical_features_df = geo_select_clinical_features(
    clinical_data, 
    trait=trait, 
    trait_row=trait_row, 
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Now link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features_df, normalized_gene_data)
print("Linked data shape:", linked_data.shape)

# Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)

# 4. Determine whether the trait and some demographic features are severely biased, and remove biased features.
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information.
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression from monocytes of rheumatoid arthritis patients, with osteoporosis status included in comorbidity information."
)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")

Normalized gene data saved to ../../output/preprocess/Chronic_Fatigue_Syndrome/gene_data/GSE67311.csv
Clinical data saved to ../../output/preprocess/Chronic_Fatigue_Syndrome/clinical_data/GSE67311.csv
Linked data shape: (142, 19855)


For the feature 'Chronic_Fatigue_Syndrome', the least common label is '1.0' with 10 occurrences. This represents 7.52% of the dataset.
The distribution of the feature 'Chronic_Fatigue_Syndrome' in this dataset is fine.



Linked data saved to ../../output/preprocess/Chronic_Fatigue_Syndrome/GSE67311.csv
